In [ ]:
# Install if needed:
# !pip install SPARQLWrapper pandas openpyxl

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

BASE = "http://127.0.0.1:5000/sparql"

# Map ontology keys → output file stems
TARGETS = {
    "bfo":  "bfo-class.xlsx",
    "ies":  "ies-class.xlsx",
    "qudt": "qudt-class.xlsx",
    "ccom": "ccom-class.xlsx",
    "ccot": "ccot-class.xlsx",
    "to":   "to-class.xlsx",
}

CLASS_LABEL_QUERY = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
SELECT ?iri ?label
WHERE {
  ?iri a ?classType .
  VALUES ?classType { owl:Class rdfs:Class }
  OPTIONAL { ?iri rdfs:label ?label }
  FILTER NOT EXISTS { ?iri owl:deprecated true }
}
ORDER BY ?label
"""

def run_select(endpoint_url: str, query: str) -> pd.DataFrame:
    sp = SPARQLWrapper(endpoint_url)
    sp.setQuery(query)
    sp.setReturnFormat(JSON)
    data = sp.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        iri = b.get("iri", {}).get("value")
        label = b.get("label", {}).get("value")
        rows.append({"iri": iri, "label": label})
    return pd.DataFrame(rows, columns=["iri", "label"])

for key, outfile in TARGETS.items():
    endpoint = f"{BASE}/{key}"
    print(f"Querying {endpoint} …")
    df = run_select(endpoint, CLASS_LABEL_QUERY)
    # Save to Excel
    df.to_excel(outfile, index=False)
    print(f"Saved {len(df)} rows to {outfile}")
